In [1]:
cd /home/jovyan/Documents/dsi_plus/Projects/project_3 - Madelon_feature_selection

/home/jovyan/Documents/dsi_plus/Projects/project_3 - Madelon_feature_selection


In [2]:
#Import standard modules
%run lib/__init__.py
%matplotlib inline

### Project 3: Madelon Dataset - Feature Selection + Classification

## Step 2. Feature Selection - SelectKbest

Hyperparatmeters: 
   * k threshold : 
       * number of features to select and pass to next method ( based on lowest pvalues). 
       * We are looking for 5 informative and 15 redundant features so we will try a range of values 
           k = [5,10,15,20]
   * scoring function: 
       * f_classifi - used for classification problems with Categorical target and continous features
       * Function used to calculate pvalues of ANOVA F-test. 
       * Low pvalue implies => reject null hypothesis
           null hypothesis = feature and tartget have not statistically different.
           
random.seed removed and shuffle split as causing model to get stuck. Stratefiedkfold is the default used by gridsearch when the last fit model is a classifier , so only the shuffle element is missing.

In [3]:
#random.seed=42

In [4]:
#Load files names for datasets
pickle_files=Pickle_files()
datafiles=pickle_files.read_datafiles()


In [5]:
#List of classifiers to use for scoring selection
Classifiers={'SKB_KNN':KNeighborsClassifier(),'SKB_LR': LogisticRegression(C=100),\
             'SKB_DT': DecisionTreeClassifier(), 'SKB_SVC':SVC()}

XClassifiers={'SKB_LR': LogisticRegression(C=1000)}

In [6]:
def skb_pipeline(ds,Classifiers, paramG,datafiles=datafiles ):
    ''' 
    Pipline to find best features using SelectFromModel with LogisticRegression estimator
    '''
    X=pd.read_pickle(datafiles[ds][2])
    y=pd.read_pickle(datafiles[ds][4])
    print(datafiles[ds][2],X.shape, datafiles[ds][4],y.shape)
    
    results=[]
    
    for n,model in tqdm(Classifiers.items()):   
       
        
        pipe=Pipeline(steps=[
                                ('scaler' , StandardScaler()),
                                ('skb' , SelectKBest()),
                                ('clf' ,model)
                               ])
    
        param=paramG

        grsCV=GridSearchCV(pipe,param,n_jobs=-1, verbose=1,\
                           cv=StratifiedShuffleSplit(n_splits=5, test_size=.2))
       
        grsCV.fit(X,y.squeeze())
        
        
        skb_support=grsCV.best_estimator_.named_steps['skb'].get_support()
        
        results_n={ 'dataset' : ds,
                    'model'  : n,
                    'best_test_score' : grsCV.best_score_,
                    'best_params' : grsCV.best_params_ , 
                    'featSelect' : np.where(skb_support)[0], 
                    'n_featSelect' : np.sum(skb_support)
                    }
        results.append(results_n)
    
        
    results_df=pd.DataFrame(results)
    
    return results_df

#### UCI Dataset

In [7]:
UCI_param=({'skb__k': [5,10,15,20]})

In [8]:
%%time
UCI_results_df=skb_pipeline('UCI',Classifiers, UCI_param)

  0%|          | 0/4 [00:00<?, ?it/s]

./data/pickle_data/UCI_X_train (2080, 500) ./data/pickle_data/UCI_y_train (2080, 1)
Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    0.7s finished
 25%|██▌       | 1/4 [00:00<00:02,  1.22it/s]

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    0.6s finished
 50%|█████     | 2/4 [00:01<00:01,  1.21it/s]

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    0.5s finished
 75%|███████▌  | 3/4 [00:02<00:00,  1.31it/s]

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    1.4s finished
100%|██████████| 4/4 [00:03<00:00,  1.01it/s]

CPU times: user 1.5 s, sys: 190 ms, total: 1.69 s
Wall time: 4.03 s


In [9]:
UCI_results_df

,best_params,best_test_score,dataset,featSelect,model,n_featSelect
0,{'skb__k': 10},0.871154,UCI,"[48, 64, 105, 128, 241, 336, 338, 378, 442, 475]",SKB_KNN,10
1,{'skb__k': 5},0.590865,UCI,"[64, 128, 241, 336, 475]",SKB_LR,5
2,{'skb__k': 15},0.787019,UCI,"[48, 64, 105, 128, 241, 323, 336, 338, 378, 38...",SKB_DT,15
3,{'skb__k': 10},0.800481,UCI,"[48, 64, 105, 128, 241, 336, 338, 378, 442, 475]",SKB_SVC,10


In [10]:
UCI_results_df.to_pickle('./data/pickle_data/UCI_bestfeatures_skb')

#### Sample 1

In [14]:
Sample1_param=({'skb__k': [5,10,15,20]})

In [15]:
%%time
Sample1_results_df =skb_pipeline('Sample1',Classifiers, Sample1_param)

  0%|          | 0/4 [00:00<?, ?it/s]

./data/pickle_data/Sample1_X_train (7364, 1000) ./data/pickle_data/Sample1_y_train (7364,)
Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    5.0s finished
 25%|██▌       | 1/4 [00:05<00:16,  5.42s/it]

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    3.7s finished
 50%|█████     | 2/4 [00:09<00:09,  4.78s/it]

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    3.2s finished
 75%|███████▌  | 3/4 [00:13<00:04,  4.41s/it]

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:   13.2s finished
100%|██████████| 4/4 [00:28<00:00,  7.17s/it]

CPU times: user 10.6 s, sys: 830 ms, total: 11.4 s
Wall time: 28.8 s


In [16]:
Sample1_results_df 

,best_params,best_test_score,dataset,featSelect,model,n_featSelect
0,{'skb__k': 15},0.808554,Sample1,"[257, 269, 315, 336, 341, 504, 526, 681, 701, ...",SKB_KNN,15
1,{'skb__k': 15},0.647794,Sample1,"[257, 269, 315, 336, 341, 504, 526, 681, 701, ...",SKB_LR,15
2,{'skb__k': 15},0.722743,Sample1,"[257, 269, 315, 336, 341, 504, 526, 681, 701, ...",SKB_DT,15
3,{'skb__k': 15},0.767142,Sample1,"[257, 269, 315, 336, 341, 504, 526, 681, 701, ...",SKB_SVC,15


#### Sample 2

In [17]:
%%time
Sample2_results_df =skb_pipeline('Sample2',Classifiers, Sample1_param)

  0%|          | 0/4 [00:00<?, ?it/s]

./data/pickle_data/Sample2_X_train (7325, 1000) ./data/pickle_data/Sample2_y_train (7325,)
Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    4.7s finished
 25%|██▌       | 1/4 [00:05<00:15,  5.14s/it]

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    3.3s finished
 50%|█████     | 2/4 [00:08<00:08,  4.45s/it]

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    3.1s finished
 75%|███████▌  | 3/4 [00:12<00:04,  4.17s/it]

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:   12.9s finished
100%|██████████| 4/4 [00:27<00:00,  6.89s/it]

CPU times: user 10.6 s, sys: 710 ms, total: 11.3 s
Wall time: 27.7 s


In [18]:
Sample2_results_df 

,best_params,best_test_score,dataset,featSelect,model,n_featSelect
0,{'skb__k': 15},0.803959,Sample2,"[257, 269, 315, 336, 341, 504, 681, 701, 724, ...",SKB_KNN,15
1,{'skb__k': 15},0.657201,Sample2,"[257, 269, 315, 336, 341, 504, 681, 701, 724, ...",SKB_LR,15
2,{'skb__k': 15},0.729147,Sample2,"[257, 269, 315, 336, 341, 504, 681, 701, 724, ...",SKB_DT,15
3,{'skb__k': 15},0.762594,Sample2,"[257, 269, 315, 336, 341, 504, 681, 701, 724, ...",SKB_SVC,15


#### Sample 3

In [19]:
%%time
Sample3_results_df =skb_pipeline('Sample3',Classifiers, Sample1_param)

  0%|          | 0/4 [00:00<?, ?it/s]

./data/pickle_data/Sample3_X_train (7300, 1000) ./data/pickle_data/Sample3_y_train (7300,)
Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    4.4s finished
 25%|██▌       | 1/4 [00:04<00:14,  4.79s/it]

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    3.3s finished
 50%|█████     | 2/4 [00:08<00:08,  4.26s/it]

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    3.0s finished
 75%|███████▌  | 3/4 [00:11<00:03,  4.00s/it]

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:   12.8s finished
100%|██████████| 4/4 [00:26<00:00,  6.68s/it]

CPU times: user 10.1 s, sys: 810 ms, total: 10.9 s
Wall time: 26.9 s


In [20]:
Sample3_results_df 

,best_params,best_test_score,dataset,featSelect,model,n_featSelect
0,{'skb__k': 10},0.811781,Sample3,"[269, 315, 341, 681, 701, 736, 769, 808, 829, ...",SKB_KNN,10
1,{'skb__k': 20},0.635753,Sample3,"[257, 269, 315, 336, 341, 395, 494, 504, 526, ...",SKB_LR,20
2,{'skb__k': 10},0.732877,Sample3,"[269, 315, 341, 681, 701, 736, 769, 808, 829, ...",SKB_DT,10
3,{'skb__k': 10},0.765890,Sample3,"[269, 315, 341, 681, 701, 736, 769, 808, 829, ...",SKB_SVC,10


#### Consolidate Sample results

In [21]:
Sample_results_df=pd.concat([Sample1_results_df,Sample2_results_df,\
                          Sample3_results_df], axis=0, ignore_index=True)

In [22]:
Sample_results_df.to_pickle('./data/pickle_data/Samples_bestfeatures_skb')